In [1]:
from openpyxl import load_workbook

In [2]:
stateAbbrMap = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
    "National": "US"
}

In [3]:
import re

def getText(sheet, row_idx, col_idx):
    text = sheet[row_idx + 1][col_idx].value.strip()
    
    return re.sub(r"(\s*)\n(\s*)", "\\n", text)


def read_sheet(sheet):
    col_end = 0
    for idx, col in enumerate(sheet.columns):
        if type(col[0].value) == str:
            if "pdf" in col[0].value.strip().lower():
                col_end = idx
                break

    state = {}
    # go through each row in a sheet
    row_idx = 1
    current = None
    for idx, row in enumerate(sheet.rows):
        if idx == 0:
            continue

        # break point here
        if idx == 30:
            break

        # go through each row within current category
        if row[0].value != None or idx == 29:
            while row_idx < idx:
                if current == None:
                    break
                
                # assign each column to current label
                col_idx = 2
                while col_idx < col_end:
                    if sheet[row_idx + 1][col_idx].value == None:
                        col_idx += 1
                        continue

                    if "not" in sheet[row_idx + 1][col_idx].value.strip().lower() and "exist" in sheet[row_idx + 1][col_idx].value.strip().lower():
                        col_idx += 1
                        continue

                    assignee = sheet[row_idx + 1][1].value.strip() if sheet[row_idx + 1][1].value != None else "Other Resources"
                    if assignee not in state[current]:
                        cell = sheet[row_idx + 1][col_idx]
                        if cell.hyperlink:
                            state[current][assignee] = [{"text": getText(sheet, row_idx, col_idx), "link": cell.hyperlink.target}] if sheet[row_idx + 1][col_idx].value != None else []
                        else:
                            state[current][assignee] = [{"text": getText(sheet, row_idx, col_idx)}] if sheet[row_idx + 1][col_idx].value != None else []
                        
                    else:
                        cell = sheet[row_idx + 1][col_idx]
                        if cell.hyperlink:
                            state[current][assignee].append({"text": getText(sheet, row_idx, col_idx), "link": cell.hyperlink.target})
                        else:
                            state[current][assignee].append({"text": getText(sheet, row_idx, col_idx)})

                    col_idx += 1
                
                row_idx += 1

            if (idx == 29):
                break

            if current != None and not state[current]:
                state.pop(current, None)

            current = row[0].value.strip()
            row_idx = idx
            state[current] = {}

    return state

In [4]:
dict = {}
workbook = load_workbook(filename="../data/states-policy-data.xlsx")
for sheet_name in workbook.sheetnames:
    if sheet_name != "SPECs" and sheet_name != "KEY":
        sheet = workbook[sheet_name]
        state = sheet_name.upper()
        dict[state] = read_sheet(sheet)

In [5]:
geoJson = {"type": "FeatureCollection", "features": []}
for key, value in stateAbbrMap.items():
    if value not in dict:
        continue
    
    geoJson["features"].append({
        "type": "Feature",
        "properties": {
            "name": key,
            "abbr": value,
            "data": dict[value]
        },
        "geometry": None
    })

In [6]:
import json

with open("../data/states-careers-policy.json", "w") as outfile:
    json.dump(geoJson, outfile)